In [ ]:
#meta 12/12/2020 Kaggle Cassava Model
#fast.ai 2020 study group Kaggle Competition
#src https://www.kaggle.com/c/cassava-leaf-disease-classification
#prev competition src https://www.kaggle.com/c/cassava-disease
#references https://arxiv.org/pdf/1908.02900.pdf

#input: kaggle files - images, csv with 5 cats
#output: data and model artifacts, no submission.csv
#pwd: /kaggle/working

#history
#12/12/2020 MY INITIAL CODE - SMALL SAMPLE
#      Make sure submission works

#12/13/2020 MY INITIAL MODEL
#      Modeling with fast.ai
#      Make sure submission works with real size

#12/28/2020 MY MODEL TO SUBMIT
#      My sumbissions didn't work
#      Simulate test files and verify multiple submission - see part Xtra
#      Get rid of submission loop
#      1st successful submit with .779 accuracy

#01/03/2021 MODEL
#      dataprep: changed to DataBlock
#      item_tfms = Resize(128), batch_tfms = aug_transforms(mult=2) Ch2,pg74
#      model: cnn_learner(dls, resnet18, metrics=accuracy)
#      learn.fine_tune(10)
#      successful submit with .770 accuracy
 
#01/03/2021 MODEL - DELTA CENTER CROP
#      dataprep delta: DataBlock: Center crop
#      src https://docs.fast.ai/vision.augment#aug_transforms
#      item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2)
#      model: cnn_learner(dls, resnet18, metrics=accuracy)
#      learn.fine_tune(10)
#      successful submit with 0.876 accuracy
 
#1/3/2020 MODEL - DELTA IMBALANCED DS -> OVERSAMPLE WITH SMOTE
#      src https://www.kaggle.com/zakraicik/fast-ai-starter-w-smote-resample
#      dataprep delta: dblk.dataloaders(df_train_tidy[['image_id_res', 'label']], bs=64)
#      item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2)
#      model: cnn_learner(dls, resnet18, metrics=accuracy)
#      learn.fine_tune(10)
#      successful submit with 0.842 accuracy
#      Note about SMOTE: not really synthetic oversampling, overly naive as X is only filename, not real input
#      Therefore SMOTE is not a good way to go as coded here.

#1/4/2020 MODEL - DELTA minus:SMOTE plus:STRATIFIED VALIDATION SET
#      dataprep delta: Stratified data split
#      src https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
#      dataprep delta: add df_train['is_valid'] column needed for ColSplitter()
#      src https://docs.fast.ai/tutorial.datablock.html
#      dataprep delta: dblock splitter=ColSplitter(), item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2)
#      model: cnn_learner(dls, resnet18, metrics=accuracy)
#      learn.fine_tune(10)
#      successful submit with 0.875 accuracy

#1/5/2020 MODEL - ADD DEBUG MODE
#      dataprep delta: in DEBUG mode, train 50% subsample
#      successful submit with 0.860 accuracy

#1/5/2020 MODEL - DELTA LOSS FN 
#      in DEBUG mode, train 50% subsample
#      model delta: src Ch7 pg251 loss_func=LabelSmoothingCrossEntropy
#      model delta: learn = cnn_learner(dls, resnet18, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
#      learn.fine_tune(10)
#      successful submit with 0.862 accuracy

#1/5/2020 MODEL - DELTA LOSS FN 
#      in FULL mode, train entire ds
#      dataprep: dblock splitter=ColSplitter(), item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2)
#      model: learn = cnn_learner(dls, resnet18, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
#      learn.fine_tune(10)
#      successful submit with 0.877 and 0.878 accuracy

#1/9/2020 MODEL - BACK TO v15 SCORE0.877, TRAIN FT10. EXPORT FOR INFERENCE
#      in FULL mode, train entire ds
#      dataprep: dblocksplitter=ColSplitter(), item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2))
#      model: learn = cnn_learner(dls, resnet18, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
#      learn.fine_tune(10)
#      export output for inference; no submission file output

#here 1/15/2020 MODEL - BACK TO v15 SCORE0.877, COMPLETED INFERENCE. REFACTOR TRAIN FT10. EXPORT FOR INFERENCE
#      in FULL mode, train entire ds, export model
#      2-infer with path_model = Path('../input/mycassavamodelv0')
#      Score: .0879  GPU submission time: not captured Rank: 1744

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import os
import time #to track performance time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#stratified split
from sklearn.model_selection import StratifiedShuffleSplit

#modeling
from fastai.vision.all import *


# Input data files are available in the read-only "../input/" directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#--constants and variables
SEED = 24
DEBUG = 0

#--functions
#define label lookup 
def lookup_label(image_id): return int(df_train[df_train['image_id']== image_id]['label'])

#test function
#lookup_label('100042118.jpg')

# Cassava Disease Classification
##### Objective: Classify pictures of cassava leaves into 1 of 4 disease categories or healthy.

As the second-largest provider of carbohydrates in Africa, cassava is a key food security crop grown by smallholder farmers because it can withstand harsh conditions. At least 80% of household farms in Sub-Saharan Africa grow this starchy root, but viral diseases are major sources of poor yields. With the help of data science, it may be possible to identify common diseases so they can be treated.

Existing methods of disease detection require farmers to solicit the help of government-funded agricultural experts to visually inspect and diagnose the plants. This suffers from being labor-intensive, low-supply and costly. As an added challenge, effective solutions for farmers must perform well under significant constraints, since African farmers may only have access to mobile-quality cameras with low-bandwidth.

In this competition, we introduce a dataset of 21,367 labeled images collected during a regular survey in Uganda. Most images were crowdsourced from farmers taking photos of their gardens, and annotated by experts at the National Crops Resources Research Institute (NaCRRI) in collaboration with the AI lab at Makerere University, Kampala. This is in a format that most realistically represents what farmers would need to diagnose in real life.

Your task is to classify each cassava image into four disease categories or a fifth category indicating a healthy leaf. With your help, farmers may be able to quickly identify diseased plants, potentially saving their crops before they inflict irreparable damage.


## 0. Load Data

- Input: a dataset of 21,367 labeled images collected during a regular survey in Uganda.  
- Output: 5 categories, including healthy + 4 fine-grained cassava leaf disease categories

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
path.ls()

### 0.1 Images  
21397 train images and 1 test image

In [ ]:
(path/'train_images').ls(), (path/'test_images').ls()

In [ ]:
#load train/test images
train_img = get_image_files(path/'train_images')
test_img = get_image_files(path/'test_images')

print("Number of files: ", (len(train_img), len(test_img)))

### 0.2 Disease Labels Mapping

In [ ]:
with open((path/'label_num_to_disease_map.json')) as json_file:
    target_map = json.load(json_file)
    
print(target_map)

### 0.3 Train Dataset

In [ ]:
df_train = pd.read_csv(path/'train.csv')

print('train data\n', df_train.shape)
display(df_train.head(6))


In [ ]:
#train subsample
if DEBUG:
    #stratified subsample
    X_debug = df_train['image_id']
    y_debug = df_train['label']
    sss_debug = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
    
    for train_idx_debug, valid_idx_debug in sss_debug.split(X_debug, y_debug):
        print("TRAIN:", train_idx_debug, "VALID:", valid_idx_debug)
        y_train_debug, y_valid_debug = y_debug[train_idx_debug], y_debug[valid_idx_debug]

    y_train_debug.value_counts().sort_index()
    y_train_debug.value_counts().sort_index().plot(kind='bar');
    

In [ ]:
#train subsample
if DEBUG:
    #update df_train with train index 
    df_train = df_train.loc[df_train.index.isin(valid_idx_debug)].reset_index()
    print(df_train.shape)
    print(df_train.head())
    

In [ ]:
print(df_train['label'].value_counts().sort_index())
df_train['label'].value_counts().sort_index().plot(kind='bar');

In [ ]:
#view an image
image_example = '100042118.jpg'
print(lookup_label(image_example))
print(path/'train_images'/image_example)
#Image.open(path/'train_images'/image_example)


## 1. Prep Data
### 1.0 Stratified Split
Stratified shuffle split to create a proportional validation set

In [ ]:
X = df_train['image_id']
y = df_train['label']
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
print(sss.get_n_splits(X, y))

In [ ]:
df_train.head()

In [ ]:
for train_idx, valid_idx in sss.split(X, y):
    print("TRAIN:", train_idx, "VALID:", valid_idx)
    y_train, y_valid = y[train_idx], y[valid_idx]

y_train.value_counts().sort_index(), y_valid.value_counts().sort_index()

In [ ]:
#vis validation ds distribution
y_valid.value_counts().sort_index().plot(kind = 'bar')

In [ ]:
#update df_train with valid index 
df_train['is_valid'] = 0
df_train.loc[df_train.index.isin(valid_idx), 'is_valid'] = 1
df_train['is_valid'].value_counts()
        


### 1.1 From Data to Dataloaders
with DataBlock

path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
def get_x(row): return path/'train_images'/row['image_id']
def get_y(row): return row['label']

In [ ]:
#dataloaders
dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_x = get_x,
                 get_y = get_y,
                 splitter=ColSplitter(),
                 item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2))

dls = dblock.dataloaders(df_train, bs=64)

len(dls.train_ds), len(dls.valid_ds)

In [ ]:
dls.train.show_batch()

## 2. Model
Learn a model to classify a given image into these 4 disease categories or a 5th category indicating a healthy leaf, using the images in the training data

### 2.0 Pretrained model offline

In [ ]:
#src Daniel
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet185c106cdepth/resnet18-5c106cde.pth' '/root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth'

In [ ]:
#if Internet enabled, download the pretrained model
#import torchvision.models as models
##arch = models.resnet18

#if Internet not enabled
#RESNET18_PATH = '/kaggle/input/resnet185c106cdepth/resnet18-5c106cde.pth'
#torch.load(resnet18, RESNET18_PATH)

### 2.1 Train 
Initial training 10 epochs.  Fine-tune after a successful submission

In [ ]:
#track time
t0 = time.time()

#Ch7 pg251 LabelSmoothingCrossEntropy()
learn = cnn_learner(dls, resnet18, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
learn.fine_tune(10)

t1 = time.time()
print ("Training time (in min)", (time.time() - t0)/60)

In [ ]:
#save the state of your model
learn.save('my-full-10epochs')
#export model
learn.export()

In [ ]:
learn.recorder.plot_loss()

### 2.2 Evaluate

In [ ]:
learn = load_learner('../input/mycassavamodelv0/export.pkl')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.most_confused(min_val=10)

In [ ]:
#use exports for inference
mystop

## 3. Predict
Predict on one file

In [ ]:
#view test images - only one
#Image.open(test_img[0])

#predict one, returns a tuple, get 0th item
y_hat = learn.predict(test_img[0])
y_hat[0]

Test dataset

In [ ]:
#create test images id

test_img_id = []

for f in test_img:
    #f_name = str(f).split("/")[-1]
    #print (f_name)
    test_img_id.append(f)

#images in test files
test_img_id

In [ ]:
test_dl = dls.test_dl(test_img_id)
len(test_dl)

## 4. Submit


In [ ]:
#template
sample_submission = pd.read_csv(path/'sample_submission.csv')
submission = sample_submission.copy()

#predict
preds,_ = learn.get_preds(dl=test_dl)
#preds

#submission
submission['image_id'] = [ str(item).split("/")[-1] for item in test_img_id]
submission['label'] = preds.argmax(dim=-1).numpy()
submission.head()


In [ ]:
submission.to_csv('submission.csv', index=False)

## Xtra

In [ ]:
#export data
torch.save(df_train, 'df_train.pkl')